# Import

In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import accuracy_score

In [2]:
from Saves.HelperFunctions import *
from Preprocessing import preprocess

In [3]:
train_data = pd.read_csv('Data/train.csv')
# test_data = pd.read_csv('Data/test.csv')
test_data = None

In [4]:
train_data.head(2)

,SampleID,Discussion,Category
0,1,"Without sitting down and doing it manually, yo...",Sports
1,2,All your Search ends with this link.,STEM


In [5]:
# test_data.head(3)

# Preprocessing

In [ ]:
pre_method = 2
X_train_padded, Y_train, X_test_padded, num_unique_words, max_sequence_length = preprocess(train_data=train_data,
                                                                                            test_data=test_data, 
                                                                                            pre_method=pre_method,
                                                                                            fx_opt=2)

Drop Nan...
	train_data.shape before (24989, 3)
	train_data.shape after (24646, 3)
start preprocessing...
Encoding Y_train...
Calc unique words...
	Num of Unique words: 59373
Tokenizer...


In [8]:
print(f"X_train.shape: {X_train_padded.shape}")
# print(f"X_test.shape: {X_test_padded.shape}")
print(f"Y_train.shape: {Y_train.shape}")

X_train.shape: (24646, 809)
Y_train.shape: (24646,)


In [9]:
Y_train.head(3)

0    1
1    4
2    4
Name: Category, dtype: int64

# RNN

In [12]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Dropout

In [14]:
print(f"num_unique_words: {num_unique_words}")
print(f"max_sequence_length: {max_sequence_length}")

num_unique_words: 59373
max_sequence_length: 809


In [20]:
num_classes = 5
# Define the model
rnn_model = models.Sequential([
    layers.Embedding(input_dim=num_unique_words, output_dim=128),
    layers.SimpleRNN(128, activation='tanh', return_sequences=False),
    layers.Dense(64, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

# Explicitly build the model
rnn_model.build(input_shape=(None, max_sequence_length))

# Compile the model
rnn_model.compile(optimizer='adam',
                    loss='categorical_crossentropy',  # For multi-class classification
                    metrics=['accuracy'])

# Summary of the model
rnn_model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (None, 809, 128)       │     7,599,744 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_2 (SimpleRNN)        │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,641,221 (29.15 MB)

 Trainable params: 7,641,221 (29.15 MB)

 Non-trainable params: 0 (0.00 B)

In [15]:
# Assuming Y_train contains labels in integer form (e.g., [0, 1, 2, 3, 4])
Y_train_categorical = to_categorical(Y_train, num_classes=num_classes)

In [17]:
epochs = 2
batch_size = 32
validation_split = 0.2

In [21]:
rnn_model.fit(X_train_padded, Y_train_categorical, epochs=epochs, batch_size=batch_size, validation_split=validation_split)

Epoch 1/2
617/617 ━━━━━━━━━━━━━━━━━━━━ 364s 573ms/step - accuracy: 0.2159 - loss: 1.6332 - val_accuracy: 0.2193 - val_loss: 1.6309
Epoch 2/2
617/617 ━━━━━━━━━━━━━━━━━━━━ 290s 470ms/step - accuracy: 0.2225 - loss: 1.6287 - val_accuracy: 0.2211 - val_loss: 1.6111


In [22]:
# Predict the labels for the test set
predictions = rnn_model.predict(X_train_padded)

# If it's a multi-class classification task, get the predicted class for each sample
predicted_classes = np.argmax(predictions, axis=1)

771/771 ━━━━━━━━━━━━━━━━━━━━ 71s 91ms/step


## Evaluation

### Train

In [ ]:
Y_train_pred = rnn_model.predict(X_train_padded)

In [27]:
train_accuracy = accuracy_score(predicted_classes, Y_train)
print(f"Train Accuracy: {train_accuracy}")

Train Accuracy: 0.22421488273959264


In [ ]:
file_name = f'RNN-m{pre_method}-e{epochs}-a{int(train_accuracy*100)}'
rnn_model.save(f'Models/RNN/{file_name}.h5')

### Test

In [ ]:
# Y_test_pred = ffnn_model.predict(X = X_test)

In [ ]:
# file_name = f'FFNN-m{pre_method}-e{epochs}'
# save_csv(data=Y_test_pred, file_name=file_name, header=['SampleID', 'Category'], numbering=True)